Let's give this contest a go. Hopefully not too much data that it overloads the Kaggle kernel...
Plan is to explore the data beforehand, and try to standardise where I can.
Then try out a bunch of different models and see what results I get.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import glob
import io
import math
import os
import matplotlib


from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV,RidgeClassifierCV,RidgeClassifier,LogisticRegression,LogisticRegressionCV
from sklearn.model_selection import cross_val_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Read all the data in.

In [ ]:
# read in the data here
train_df = pd.read_csv("../input/train.csv",sep=",")
test_df = pd.read_csv("../input/test.csv",sep=",")

Combine the dataframes together for cleaning.

In [ ]:
#Combine the dataframes
combined_df=pd.DataFrame()
combined_df=pd.concat([train_df,test_df])

Looks like the data set is complete and all numerical. Since it's all anonymized, it's not clear what anything means.  To start with, I'll separate out the Target & ID, so I can start to deal with the numerical data.

In [ ]:
num_df=pd.DataFrame.copy(combined_df)

#Put the Target & ID aside for now
target_df=pd.DataFrame()
target_df["TARGET"]=num_df["TARGET"]
num_df=num_df.drop(["TARGET"],axis=1)

#Same with ID
ID_df=pd.DataFrame()
ID_df["ID"]=num_df["ID"]
num_df=num_df.drop(["ID"],axis=1)

One problem is what to do with the huge values in some columns. It seems like -999999 is an error code in var3, which most people replace with 2. 9999999999 appears a lot in the data, and I need to decide how to deal with it. Some people seem to encode it as nan, although I might get away with changing it to another high value, but outside the range of the rest of the data (like 10/100). For now, I'm hoping converting to 100 is enough.

In [ ]:
#Replaces huge -ve number with 2.
clean_df=pd.DataFrame.copy(num_df)
clean_df["var3"].value_counts()
clean_df.loc[clean_df['var3']==-999999,'var3']=2
clean_df["var3"].value_counts()

In [ ]:
clean_df[num_df==9999999999] = 100
clean_df.skew().sort_values(ascending=False)

Try to normalize the data a bit. I want to remove the columns which contain all zeros, then try log transforming some others if they are highly skewed.

In [ ]:
#Remove all fully zero columns

clean_df=clean_df.loc[:, (clean_df != 0).any(axis=0)]
clean_df.shape

In [ ]:
skewed_feats= clean_df.skew()
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

unskewed_feats= clean_df.skew()
unskewed_feats = unskewed_feats[unskewed_feats < 0.75]
unskewed_feats = unskewed_feats.index

skewed_feats,unskewed_feats

#transform_df=pd.DataFrame.copy(clean_df)
#transform_df[unskewed_feats]=(No_sparse_df[unskewed_feats]
#                               - No_sparse_df[unskewed_feats].mean()) / (No_sparse_df[unskewed_feats].max() - No_sparse_df[unskewed_feats].min())
#transform_df[skewed_feats] = np.log1p(No_sparse_df[skewed_feats])

transform_df=pd.DataFrame.copy(clean_df)
transform_df=(transform_df-transform_df.mean()) / (transform_df.max() - transform_df.min())
transform_df[skewed_feats] = np.log1p(transform_df[skewed_feats])

transform_df["imp_reemb_var17_hace3"].describe()

#_ = transform_df.hist(bins=40, figsize=(18, 18), xlabelsize=10)
#transform_df.skew().sort_values(ascending=False)
#transform_df.describe()

Combine the dataframes back.


In [ ]:
Eng_df=pd.DataFrame()

#transform_df
Eng_df=pd.concat([transform_df,target_df,ID_df], axis=1)

#Split back into the original data

Eng_train_df=pd.DataFrame()
Eng_train_df=Eng_df.head(76020)
Eng_test_df=pd.DataFrame()
Eng_test_df=Eng_df.tail(75818)


# shuffle the data, just in case
Eng_train_df = Eng_train_df.reindex(np.random.permutation(Eng_train_df.index))
Eng_train_df.describe()

In [ ]:
# Process features and target
def preprocess_features(training_df):
 
  selected_features = training_df[training_df.columns.difference(["TARGET","ID"])]
  processed_features = selected_features.copy()
  return processed_features

def preprocess_targets(training_df):
 
  output_targets = pd.DataFrame()
    
  output_targets["TARGET"] = (
    training_df["TARGET"])
  return output_targets

Getting the training / test inputs.

In [ ]:
training_examples= preprocess_features(Eng_train_df)
test_examples = preprocess_features(Eng_test_df)
training_targets = preprocess_targets(Eng_train_df)

To start off with, I want to try out a Ridge Classifier model to work, and see what the best auc I can get is.
For now, all I've done is remove the fully zero columns and try to normalise the data.

In [ ]:
#Let's just try ridge out of the box on this set, and alter alpha
#Ridge=RidgeCV()
#crossval_scores = cross_val_score(Ridge, training_examples, np.ravel(training_targets), scoring='roc_auc', cv=10)
#crossval_scores.mean()
#Default = 0.6993807210214967
#After dropping some values, I get:#0.7896482020577332
#Dropping just zero columns: 0.7256913010855297
#Sparsity cut-off at 100: 0.789270318386544
#500: 0.7888133742542437
#1k:0.7896846933393219
#5k: 0.7910464037042653 + normalise + some log _> 0.7945947795221165
#10k: 0.7896401137512984
#10%: 0.789513131590636
#25%: 0.7845316876767147
#5k was best of this lot, but I'm not sure numbers above 100 mean much.
#1% & normalization : 0.7954232890589281
#Cut off zeros, replace the huge numbers (99999 etc), try to normalize then try with cv=3:0.7957032628326774

In [ ]:
#model_ridge=RidgeClassifierCV(scoring='roc_auc', cv=3)
#model_ridge=RidgeCV(scoring='roc_auc', cv=3)
#model_ridge.fit(training_examples, np.ravel(training_targets))
#ridge_preds = model_ridge.predict(test_examples)

#solution = pd.DataFrame({"ID":test_df["ID"], "Target":np.abs(ridge_preds)})
#solution.to_csv("Ridge.csv", index = False)
#This gets me ~.794. Still pretty bad, but much better! Seems like I shouldn't be using a classifier.

The ridge model got: 0.777054 / 0.794122 for private / public in the end.

Next up is Lasso.

In [ ]:
#Lasso=LassoCV()
#crossval_scores = cross_val_score(Lasso, training_examples, np.ravel(training_targets), scoring='roc_auc', cv=5)
#crossval_scores.mean()
#cv 3: 0.7940033210294137
#cv 5: 0.794090160348801
#cv 10: 0.7944051812300171

In [ ]:
model_lasso = LassoCV(cv=10).fit(training_examples, np.ravel(training_targets))
coef = pd.Series(model_lasso.coef_, index = training_examples.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
lasso_preds = model_lasso.predict(test_examples)

solution = pd.DataFrame({"ID":test_df["ID"], "Target":np.abs(lasso_preds)})
solution.to_csv("Lasso.csv", index = False)

Lasso manages: 0.774361 / 0.794044 on private / public, which is actually worse than ridge!

Since Lasso sets coefficients to zero, I could use it to select feature for further work. No guarantee this will do anything, but it's worth a shot!
The 61 surviving features are stored below.

In [ ]:
lasso_features = coef.iloc[coef.nonzero()[0]].sort_values()
lasso_features=lasso_features.index
lasso_features

For my final basic model,  I'm going to try xgboost on the data, and see how well it performs. Then try xgboost again, but with the features leftover from lasso.

In [ ]:
#xgb_model=xgb.XGBClassifier()
#crossval_scores = cross_val_score(xgb_model, training_examples, np.ravel(training_targets), scoring='roc_auc', cv=5)
#crossval_scores.mean()
#crossval is 0.837819894645197 if I don't do anything to the settings.

In [ ]:
#n_estimators=360, max_depth=2, learning_rate=0.1
model_xgb=xgb.XGBClassifier()
model_xgb.fit(training_examples, np.ravel(training_targets))
xgb_preds = model_xgb.predict_proba(test_examples)


solution = pd.DataFrame({"ID":test_df["ID"], "Target":np.abs(xgb_preds[:,1])})
solution.to_csv("XGB_v1.csv", index = False)

XGB using all the features gives a score of : 0.823318 / 0.836969 on private / public.

In [ ]:
#xgb_model2=xgb.XGBClassifier()
#crossval_scores = cross_val_score(xgb_model2, training_examples[lasso_features], np.ravel(training_targets), scoring='roc_auc', cv=5)
#crossval_scores.mean()
#auc score is 0.8371358886285363 without tuning params. Apparently a bit worse!

In [ ]:
#n_estimators=360, max_depth=2, learning_rate=0.1
model_xgb2=xgb.XGBClassifier()
model_xgb2.fit(training_examples[lasso_features], np.ravel(training_targets))
xgb_preds2 = model_xgb2.predict_proba(test_examples[lasso_features])
solution = pd.DataFrame({"ID":test_df["ID"], "Target":np.abs(xgb_preds2[:,1])})
solution.to_csv("XGB_v2.csv", index = False)

In [ ]:
xgb_preds2[:,1]

XGB using just the lasso derived features gives a score of : 0.820866 / 0.835203.
The private value is close to my CV value.
Looking over the actual competition, what I've learnt is that assessing your score on the public leaderboard can be dangerous!
This would have utterly failed there, but actually beaten the best score in the private one. Overfitting happened!
I have seen a few people push further, closer to .084, but not sure that's worth it.